## NDBC module
* https://supermanzer.github.io/NDBC/html/NDBC.html
* https://www.ndbc.noaa.gov/data/climatic/42019.txt
* https://www.ndbc.noaa.gov/faq/measdes.shtml

- **WDIR**: Wind direction (the direction the wind is coming from in degrees clockwise from true N) during the same period used for WSPD. See Wind Averaging Methods.
- **WSPD**: Wind speed (m/s) averaged over an eight-minute period for buoys and a two-minute period for land stations. Reported Hourly. See Wind Averaging Methods.
- **GST**: Peak 5 or 8 second gust speed (m/s) measured during the eight-minute or two-minute period. The 5 or 8 second period can be determined by payload. See the Sensor Reporting, Sampling, and Accuracy section.
- **WVHT**: Significant wave height (meters) is calculated as the average of the highest one-third of all of the wave heights during the 20-minute sampling period. See the Wave Measurements section.
- **DPD**: Dominant wave period (seconds) is the period with the maximum wave energy. See the Wave Measurements section.
- **APD**: Average wave period (seconds) of all waves during the 20-minute period. See the Wave Measurements section.
- **MWD**: The direction from which the waves at the dominant period (DPD) are coming. The units are degrees from true North, increasing clockwise, with North as 0 (zero) degrees and East as 90 degrees. See the Wave Measurements section.
- **PRES**: Sea level pressure (hPa). For C-MAN sites and Great Lakes buoys, the recorded pressure is reduced to sea level using the method described in NWS Technical Procedures Bulletin 291 (11/14/80). (Labeled BAR in Historical files)
- **ATMP**: Air temperature (Celsius). For sensor heights on buoys, see Hull Descriptions. For sensor heights at C-MAN stations, see C-MAN Sensor Locations.
- **WTMP**: Sea surface temperature (Celsius). For buoys, the depth is referenced to the hull's waterline. For fixed platforms, it varies with tide but is referenced to, or near Mean Lower Low Water (MLLW).
- **DEWP**: Dewpoint temperature taken at the same height as the air temperature measurement.
- **VIS**: Station visibility (nautical miles). Note that buoy stations are limited to reports from 0 to 1.6 nmi.
- **PTDY**: Pressure Tendency is the direction (plus or minus) and the amount of pressure change (hPa) for a three-hour period ending at the time of observation. (Not in Historical files)
- **TIDE**: The water level in feet above or below Mean Lower Low Water (MLLW).


In [ ]:
# Install nbformat
import nbformat
from IPython import get_ipython

# Define the path to the notebook
notebook_path = 'utils.ipynb'

# Read the notebook
with open(notebook_path, 'r') as file:
    nb = nbformat.read(file, as_version=4)

# Execute cells
for cell in nb.cells:
    if cell.cell_type == 'code':
        code = cell.source
        get_ipython().run_cell(code)


In [ ]:
# df_WaveDataStation42002 = pd.read_excel('WaveDataStation42002.xlsx', index_col='Datetime')
# df_WaveDataStation42019 = pd.read_excel('WaveDataStation42019.xlsx', index_col='Datetime')
df_WaveDataStation42020 = pd.read_excel('WaveDataStation2020.xlsx', index_col='Datetime')


In [ ]:
def rename_columns(df):
    df = df.rename(columns={
        'WD': 'wind_direction',
        'WDIR': 'wind_direction',
        'WSPD': 'wind_speed',
        'GST': 'wind_gust',
        'WVHT': 'wave_height',
        'DPD': 'dominant_wave_period',
        'APD': 'average_wave_period',
        'MWD': 'wave_direction',
        'PRES': 'pressure',
        'ATMP': 'air_temperature',
        'WTMP': 'water_temperature',
        'DEWP': 'dewpoint',
        'VIS': 'visibility',
        'BAR': 'sea_level_pressure',
        'PTDY': '3hr_pressure_tendency',
        'TIDE': 'water_level_above_mean'
    })
    return df


In [ ]:
df_WaveDataStation42002 = rename_columns(df_WaveDataStation42002)
df_WaveDataStation42019 = rename_columns(df_WaveDataStation42019)
df_WaveDataStation42020 = rename_columns(df_WaveDataStation42020)

In [ ]:
col_units = {'wind_direction': 'degrees',
                     'wind_speed': 'meters/second',
                     'wind_gust': 'meters/second',
                     'wave_height': 'meters',
                     'dominant_wave_period': 'seconds',
                     'average_wave_period': 'seconds',
                     'dominant_wave_direction': 'degrees',
                     'pressure': 'hPa',
                     'air_temperature': 'degC',
                     'water_temperature': 'degC',
                     'dewpoint': 'degC',
                     'visibility': 'nautical_mile',
                     '3hr_pressure_tendency': 'hPa',
                     'water_level_above_mean': 'feet',
                     'time': None}

In [ ]:
base_path= os.getcwd()

datasets = {
    "WaveDataStation42002": df_WaveDataStation42002,
    "WaveDataStation42019": df_WaveDataStation42019,
    "WaveDataStation42020": df_WaveDataStation42020
}

# column_pairs = [
#     ('wind_direction', 'wind_speed'),
#     ('mean_wave_direction', 'wave_height')
# ]

column_pairs = [
    ('wind_direction', 'wind_speed'),
    ('wave_direction', 'wave_height')
]




# Generate plots for each dataset and column pair
for subfolder, df in datasets.items():
    for direction_column, magnitude_column in column_pairs:
        # Title for the plot
        title = f"{magnitude_column.capitalize()} vs. {direction_column.capitalize()} for {subfolder}"
        
        plot_windrose_facetgrid(df, 
                            direction_column=direction_column,
                            magnitude_column=magnitude_column,
                            title=title,
                            base_path=base_path,
                            subfolder=subfolder)


In [ ]:

column_labels = {
    "wind_speed": "Wind Speed (m/s)",
    "wave_height": "Wave Height (m)",
    "average_wave_period": "Average Wave Period (s)",
    "air_temperature": "Air Temperature (°C)"
}


In [ ]:
base_path=os.getcwd()

# List of datasets
datasets = {
    "WaveDataStation42002": df_WaveDataStation42002,
    # "WaveDataStation42019": df_WaveDataStation42019,
    # "WaveDataStation42020": df_WaveDataStation42020
}

# List of magnitude columns and their corresponding titles
columns = {
    "wind_speed": "Wind Speed",
    "wave_height": "Wave Height",
    "average_wave_period": "Average Wave Period",
    "air_temperature": "Air Temperature"
}


# Generate plots for each dataset and magnitude column
for subfolder, df in datasets.items():
    for col, title_part in columns.items():
        # Title for KDE, ECDF, and Time Series plots
        title = f"Annual Distribution of {title_part} for {subfolder}"

        # KDE Plot
        plot_kde_facetgrid(
            df,
            magnitude_column=col,
            title=title,
            base_path=base_path,
            subfolder=subfolder
        )
        # ECDF Plot
        plot_ecdf_facetgrid(
            df,
            magnitude_column=col,
            title=title,
            base_path=base_path,
            subfolder=subfolder
        )
        # Time Series Plot
        plot_timeseries_facetgrid(
            df,
            magnitude_column=col,
            title=f"Annual Time Series of {title_part} for {subfolder}",
            base_path=base_path,
            subfolder=subfolder
        )


In [ ]:

# List of datasets
datasets = {
    "WaveDataStation42002": df_WaveDataStation42002,
    # "WaveDataStation42019": df_WaveDataStation42019,
    # "WaveDataStation42020": df_WaveDataStation42020
}

# List of column pairs for the plot_kde function
column_pairs = [
    ('wind_direction', 'wind_speed'),
    ('wave_direction', 'wave_height')
]

# Generate plots for each dataset and column pair
for subfolder, df in datasets.items():
    for direction_column, magnitude_column in column_pairs:
        # Title for the plot
        title = f"{magnitude_column.capitalize()} vs. {direction_column.capitalize()} for {subfolder}"

        # Year to focus on in the plot
        year = 2016  # Example year, modify as needed

        # KDE Plot
        plot_kde(
            df,
            direction_column=direction_column,
            magnitude_column=magnitude_column,
            year=year,
            title=title,
            base_path=base_path,
            subfolder=subfolder
        )
